In [1]:
#### variaveis e Importações ####
import os
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import urllib3
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image
import gc

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning) #### Desabilitar avisos de segurança do requests

HEADERS = {"User-Agent": "Mozilla/5.0"}


#### Criar pasta de imagens, se não existir
image_folder = "images"
os.makedirs(image_folder, exist_ok=True)
#### Nome do arquivo Excel
excel_file = "cards_data.xlsx"
 
 #### Colunas do DataFrame
columns = ["Nome", "Cost", "Total Cost", "Type", "P", "T", "Printings", "Variações", #### parte que vem do site compacto
          "Card Text:", "Flavor Text:", "Rarity:", "Artist:", "Link:", "Image Path", "Image URL", "Image Name", "id", #### parte que vem da pagina da carta
          "Nome_A", "Cost_A", "Total Cost_A", "Type_A", "Card Text_A", "Card Number", "Expansion"] #### parte que vem das cartas com 2 funções

#### Carregar IDs existentes, se o arquivo já existir
existing_ids = {}
if os.path.exists(excel_file):
  df_existing = pd.read_excel(excel_file, usecols=["id"])
  existing_ids = df_existing["id"].fillna(0).astype(int).tolist()  #### Converter para lista de inteiros



In [ ]:
#### def salvar_imagem(soup, headers): ####

def salvar_imagem(soup, headers):
  # global img_path, img_url, img_name, numero
  #### Criar pasta de imagens, se não existir
  image_folder = "images"
  os.makedirs(image_folder, exist_ok=True)

  card_image_div = soup.find("div", class_="cardImage")
  card_image_div_castOff = soup.find("table", class_="cardDetails cardComponent")

  if card_image_div:
    img_tag = card_image_div.find("img")
  elif card_image_div_castOff:
    img_tag = card_image_div_castOff.find("img")


  if img_tag:
    img_url = img_tag["src"]
    img_url = img_url.replace("../../", "https://gatherer.wizards.com/") 
    img_name = img_tag["alt"].replace(" ", "_").replace("//","&").replace("\"","&").replace(": ","-") + "-" + setName + "-" + numero + ".jpg"
    img_path = os.path.join(image_folder, img_name)

    #### Baixar e salvar a imagem
    img_response = requests.get(img_url, headers=headers, verify=False, stream=True)
    if img_response.status_code == 200:
      with open(img_path, "wb") as img_file:
        for chunk in img_response.iter_content(chunk_size=8192):  # Baixa e escreve em pedaços de 8KB
          img_file.write(chunk)
    else:
      print("Erro ao baixar a imagem.")
      
  dados_imagem = {"Image Path": img_path, "Image URL": img_url, "Image Name": img_name, "Number": numero} 
  return dados_imagem

In [ ]:
#### def obter_dados_cards(setUrl, Variation): #### return extracted_data, extracted_data2
def obter_dados_cards(setUrl, Variation):
  session2 = None  # Inicializa a variável
  session3 = None  # Inicializa a variável

  #### URL do card
  url = f"{setUrl}"
  data = [""] * 14  #### Inicializa a lista com 8 elementos vazios
  extracted_data = {}
  extracted_data2 = {}
  image_data = {}

  #### Fazer requisição HTTP
  session2 = requests.Session()
  session3 = None

  #### Acessa a pagina do card
  response = session2.get(url, headers=HEADERS, verify=False)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")

    #### Encontrar variações do card
    card_variations_div = soup.find("div", class_="variations")
    if card_variations_div:
      variations = card_variations_div.find_all("a")   
      if len(variations) > 1:           
        variation_url = variations[Variation - 1]["href"]
        variation_url = variation_url.replace("/Pages", "https://gatherer.wizards.com/Pages")
        url = variation_url

        #### Fazer requisição HTTP para a variação
        session3 = requests.Session()
        response = session3.get(url, headers=HEADERS, verify=False)

    session2.close()

  #### Se existe mais de uma variação, verifica qual a pagina da variação 1, 2 etc na sequencia
  #### Pois as vezes a variação 1 não é a 1ª na ordem alfabetica
  if response.status_code == 200:
    #### Para pegar o id da Carta
    match = re.search(r"multiverseid=(\d+)", url)
    numero = match.group(1)

    soup = BeautifulSoup(response.text, "html.parser")

    image_data = salvar_imagem(soup, HEADERS)
    
    #### Sem CastOff
    table_div = soup.find("div", class_="smallGreyMono", style="margin-top: 5px;")
    if table_div:
      rows = table_div.find_all("div", class_="label")
      values = table_div.find_all("div", class_="value")
      extracted_data = {row.text.strip(): value.text.strip() 
                        for row, value in zip(rows, values)}


    #### Com CastOff
    table_div_cast = soup.find_all("div", class_="smallGreyMono", style=None)
    counterA = 0
    for table_div in table_div_cast:
      # table_div = table_div_cast[0]
      if table_div and counterA == 0:
        rows = table_div.find_all("div", class_="label")
        values = table_div.find_all("div", class_="value")
        extracted_data = {row.text.strip(): value.text.strip() 
                          for row, value in zip(rows, values)}

      if table_div and counterA == 1:
        rows = table_div.find_all("div", class_="label")
        values = table_div.find_all("div", class_="value")
        extracted_data2 = {row.text.strip(): value.text.strip() 
                           for row, value in zip(rows, values)}

      print(f"extracted_data: {extracted_data}")
      print(f"extracted_data2: {extracted_data2}")    
      print(f"image_data: {image_data}")
      counterA += 1


  else:
    print("Erro ao acessar a página.")

  if session2 == None:
    session3.close()
  else:
    session2.close()
    extracted_data2 = extracted_data2.append(extracted_data, ignore_index=True)
  return   extracted_data2


In [4]:
#### def salvar_em_excel(lista_cards): ####
def salvar_em_excel(extracted_data):

  df_novo = pd.DataFrame([extracted_data])

  try:
    # Adicionando ao arquivo Excel existente
    with pd.ExcelWriter(excel_file, mode="a", if_sheet_exists="overlay", engine="openpyxl") as writer:
      df_novo.to_excel(writer, index=False, header=False)

  except Exception as e:
            print(f"Erro inesperado na linha {index}: {e}")


In [5]:
#### def dados_da_pagina_principal(soup, existing_ids): ####
def dados_da_pagina_principal(soup, existing_ids):
  lista_dados = []  # Lista para armazenar os dicionários

  for row in soup.find_all("tr"):
    cols = row.find_all("td")
      
    if not cols:
      continue  # Evita erros com linhas sem colunas
    
    name = cols[0].text.strip()
    if name in ["Name", None]: 
      continue
    
    if int(numero) in existing_ids:
      print("p", end="") # imprimi p se o item ja existe no excel
      continue

    link_tag = cols[0].find("a")
    link = link_tag["href"] if link_tag else ""
    link = link.replace("../", "https://gatherer.wizards.com/Pages/")
    
    match = re.search(r"multiverseid=(\d+)", link)
    numero = match.group(1) if match else None

    qtdImages = len(cols[5].find_all("img"))

    extracted_data = {
      "Nome": name,
      "Cost": ", ".join(img["alt"] for img in cols[1].find_all("img")),
      "Total Cost": len(cols[1].find_all("img")),
      "Type": cols[2].text.strip(),
      "P": cols[3].text.strip(),
      "T": cols[4].text.strip(),
      "Printings": ", ".join(img["alt"] for img in cols[5].find_all("img"))
    }
  
    lista_dados.append(extracted_data)  # Adiciona o dicionário à lista


    for i in range(0, qtdImages ):
      print(i, end="")
      card= []
      extracted_data2 = obter_dados_cards(link, i)
      lista_dados.append(extracted_data2)

  return lista_dados

In [ ]:
#### def processar_cada_linha(existing_ids, soup): ####
def processar_cada_linha(existing_ids, soup):

  lista_dados = []  # Lista para armazenar os dicionários

  for row in soup.find_all("tr"):
    cols = row.find_all("td")
      
    if not cols:
      continue  # Evita erros com linhas sem colunas
    
    name = cols[0].text.strip()
    if name in ["Name", None]: 
      continue

    link_tag = cols[0].find("a")
    link = link_tag["href"] if link_tag else ""
    link = link.replace("../", "https://gatherer.wizards.com/Pages/")
    
    match = re.search(r"multiverseid=(\d+)", link)
    numero = match.group(1) if match else None
    
    if int(numero) in existing_ids:
      print("p", end="") # imprimi p se o item ja existe no excel
      continue

    extracted_data = {
      "Nome": name,
      "Cost": ", ".join(img["alt"] for img in cols[1].find_all("img")),
      "Total Cost": len(cols[1].find_all("img")),
      "Type": cols[2].text.strip(),
      "P": cols[3].text.strip(),
      "T": cols[4].text.strip(),
      "Printings": ", ".join(img["alt"] for img in cols[5].find_all("img"))
    }
    
    lista_dados.append(extracted_data)  # Adiciona o dicionário à lista
    print(f"Principal: {lista_dados}")

    qtdImages = len(cols[5].find_all("img")) # Total de variações
    for i in range(0, qtdImages ):
      print(i, end="")
      
      #### pega dados da variação, se houver
      extracted_data2 = obter_dados_cards(link, i)
      lista_dados.append(extracted_data2)
      print(f"Variação {i}: {lista_dados}")

      #### salva no Excel
      salvar_em_excel(lista_dados)
      
      # adicionar o id à lista de ids existentes
      try:
        existing_ids.append(int(numero))
      except:
        print(f"Erro ao converter o ID {numero} para inteiro.")        
        continue

In [7]:
#### def processar(setName): ####
def processar(setName):
  #### Carrega a página com a lista de cards
  page = 0
  formatted_setName = setName.replace(" ", "+")  #### Substituir espaços por '+'
  url = f"https://gatherer.wizards.com/Pages/Search/Default.aspx?sort=name+&page={page}&output=compact&set=[%22{formatted_setName}%22]"

  #### Fazer requisição para a página
  session0 = requests.Session()
  response = session0.get(url, verify=False)

  #### Se a resposta não for válida, interrompe o loop
  if response.status_code == 200:  
    soup = BeautifulSoup(response.text, "html.parser")

    #### Encontrar a div com a classe "pagincontrols"
    pagin_controls = soup.find("div", class_="pagingcontrols")
    if pagin_controls:
      links = pagin_controls.find_all("a")
      qtdPages = len(links)  #### -1 para não contar o link "Next" e começar com 0

      if qtdPages <= 1: 
        qtdPages = 1  
      else:
        qtdPages = qtdPages - 1
    session0.close()

  #### Ajuste o número de páginas conforme necessário
  for page in range(0, qtdPages):

    # Carrega a pagina com links para cada versão do card
    url = f"https://gatherer.wizards.com/Pages/Search/Default.aspx?sort=name+&page={page}&output=compact&set=[%22{formatted_setName}%22]"

    #### Refazer requisição para a página
    session1 = requests.Session()
    response = session1.get(url, verify=False)

    #### Adicionar um pequeno atraso entre as requisições      
    time.sleep(2)  

    #### Se a resposta não for válida, interrompe o loop
    if response.status_code == 200:  
      soup = BeautifulSoup(response.text, "html.parser")
      contador = 0

      print(f"\n    {setName} - página {page + 1} de {qtdPages}")
      print("123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.123456789.1234567890123456789.123456789.123456789.123456789.123456789.")

      #### Lê cada linha da pagina no formato compacto      
      processar_cada_linha(existing_ids, soup)

    session1.close()  #### Fechar a sessão do requests
    gc.collect()

    # Reiniciar Python após processar um set
    print("\nReiniciando o Python")
    quit()
    os.system("MTG v2.ipynb")
  # fim do for page in range(0, qtdPages):

  

In [ ]:
# Arquivo contendo a lista original de sets
sets_file = "sets.txt"
processed_file = "processed_sets.txt" # Arquivo para armazenar os sets já processados
with open(sets_file, "r", encoding="utf-8") as f: # Ler lista original e os já processados
    sets_list = [line.strip() for line in f.readlines()]
if os.path.exists(processed_file):
    with open(processed_file, "r", encoding="utf-8") as f:
        processed_list = {line.strip() for line in f.readlines()}
else:
    processed_list = set()
# Filtrar sets que ainda não foram processados
remaining_sets = [set_name for set_name in sets_list if set_name not in processed_list]
setName = remaining_sets[0] if remaining_sets else None

# Se houver sets restantes, processar o primeiro e salvar no arquivo
if remaining_sets:
    processar(setName)
  
    # Registrar o set como processado
    with open(processed_file, "a", encoding="utf-8") as f:
        f.write(setName + "\n")

    # Reiniciar Python após processar um set
    # print("\nReiniciando o Python")
    # exit()
    # os.system("MTG v2.ipynb")
else:
  print("Todos os sets foram processados!")
